In [8]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage.interpolation import zoom
import zlib
import blosc

In [9]:
def Depth2Gray(d4d):
        d4d = 255 - cv2.cvtColor(d4d, cv2.COLOR_GRAY2RGB)
        return d4d

## Easy Intra-frame Compression Methods

### Test Compression Tools for Depth and Color Images

In [10]:
def depth_compress(i):
    name = "../pic/depth_"
    filename = name+str(i)+".png"
    depth = cv2.imread(filename)
    
    return len(depth.tostring()), len(zlib.compress(depth.tostring())), len(blosc.compress(depth.tostring()))
def color_compress(i):
    name = "../pic/rgb_"
    filename = name+str(i)+".png"
    color = cv2.imread(filename)
    return len(color.tostring()), len(zlib.compress(color.tostring())), len(blosc.compress(color.tostring()))

In [53]:
td=zd=bd=tc=zc=bc=[]
tdv = zdv = bdv = tcv = zcv = bcv= 0
for i in range(1, 441):
    t,z,b = depth_compress(i)
    td.append(t)
    zd.append(z)
    bd.append(b)
    t,z,b = color_compress(i)
    tc.append(t)
    zc.append(z)
    bc.append(b)
tdv = np.sum(td)/440.0
bdv = np.sum(bd)/440.0
zdv = np.sum(zd)/440.0
tcv = np.sum(tc)/440.0
zcv = np.sum(zc)/440.0
bcv = np.sum(bc)/440.0
print("Depth:")
print("Average Date size:",tdv)
print("After Native Zlib Compression:",zdv , 
      "; Compression Rate:",zdv/tdv*100)
print("After Blosc Compression:", bdv, 
      "; Compression Rate:",bdv/tdv*100)
print("Color:")
print("Average Data size: ", tcv)
print("After Native Zlib Compression:",zc , 
      "; Compression Rate:",zcv/tcv*100)
print("After Blosc Compression:", bcv, 
      "; Compression Rate:",bcv/tcv*100)

KeyboardInterrupt: 

### Test the compression rate if transmit them together

In [15]:
def rgbd_compress(i):
    name = "../pic/depth_"
    filename = name+str(i)+".png"
    depth = cv2.imread(filename)
    name = "../pic/rgb_"
    filename = name+str(i)+".png"
    color = cv2.imread(filename)
    rgbd = np.dstack((color, depth))
    return len(rgbd.tostring()), len(zlib.compress(rgbd.tostring())), len(blosc.compress(rgbd.tostring()))
td=zd=bd = 0
for i in range(1, 441):
    t,z,b = rgbd_compress(i)
    td += t
    zd += z
    bd += b
td /= 440.0
bd /= 440.0
zd /= 440.0
print("RGBD:")
print("Average Date size：",td)
print("After Native Zlib Compression:",zd , 
      "; Compression Rate:",zd/td*100)
print("After Blosc Compression:", bd, 
      "; Compression Rate:",bd/td*100)

RGBD:
('Average Date size\xef\xbc\x9a', 1843200.0)
('After Native Zlib Compression:', 903474.1568181819, '; Compression Rate:', 49.01661007043087)
('After Blosc Compression:', 1553262.7272727273, '; Compression Rate:', 84.26989622790404)


### Lossy Intra-frame Compression

In [18]:
def depth_compress_lossy(i):
    name = "../pic/depth_"
    filename = name+str(i)+".png"
    depth = cv2.imread(filename)
    depth = zoom(depth, [0.5,0.5,1])
    n_depth = np.mean((zoom(depth, [2,2,1]) - cv2.imread(filename)))
    return len(depth.tostring()), len(zlib.compress(depth.tostring())), len(blosc.compress(depth.tostring())), n_depth
def color_compress_lossy(i):
    name = "../pic/rgb_"
    filename = name+str(i)+".png"
    color = cv2.imread(filename)
    color = zoom(color,[0.5,0.5,1])
    n_color = np.mean((zoom(color,[2,2,1]) - cv2.imread(filename)))
    return len(color.tostring()), len(zlib.compress(color.tostring())), len(blosc.compress(color.tostring())), n_color

In [19]:
td_l=zd_l=bd_l=tc_l=zc_l=bc_l=nd=nc = 0
for i in range(1, 441):
    t,z,b,n = depth_compress_lossy(i)
    td_l += t
    zd_l += z
    bd_l += b
    nd += n
    t,z,b,n = color_compress_lossy(i)
    tc_l += t
    zc_l += z
    bc_l += b
    nc += n
td_l /= 440.0
bd_l /= 440.0
zd_l /= 440.0
tc_l /= 440.0
zc_l /= 440.0
bc_l /= 440.0
nd /= 440.0
nc /= 440.0
print("Depth:")
print("Average Date size：",td_l)
print("After Native Zlib Compression:",zd_l , 
      "; Compression Rate:",zd_l/td_l*100)
print("After Blosc Compression:", bd_l, 
      "; Compression Rate:",bd_l/td_l*100)
print("Lossy rate: Std of difference: ", nd)
print("Color:")
print("Average Data size: ", tc_l)
print("After Native Zlib Compression:",zc_l , 
      "; Compression Rate:",zc_l/tc_l*100)
print("After Blosc Compression:", bc_l, 
      "; Compression Rate:",bc_l/tc_l*100)
print("Lossy rate: Std of difference: ", nc)

Depth:
('Average Date size\xef\xbc\x9a', 230400.0)
('After Native Zlib Compression:', 24243.520454545454, '; Compression Rate:', 10.522361308396464)
('After Blosc Compression:', 93094.3159090909, '; Compression Rate:', 40.40551905776515)
('Lossy rate: Std of difference: ', 67.122543548529578)
Color:
('Average Data size: ', 230400.0)
('After Native Zlib Compression:', 197909.59772727272, '; Compression Rate:', 85.89826290246212)
('After Blosc Compression:', 228345.7431818182, '; Compression Rate:', 99.1083954782197)
('Lossy rate: Std of difference: ', 120.55603492170779)


###  JPEG Intra-frame Compression (Lossy)

In [39]:
from PIL import Image
from io import BytesIO
import StringIO

In [40]:
def getData(i):
    name = "../pic/depth_"
    filename = name+str(i)+".png"
    depth = cv2.imread(filename)
    
    name = "../pic/rgb_"
    filename = name+str(i)+".png"
    color = cv2.imread(filename)
    return np.fromstring(depth, np.uint8).reshape(480,640,3), np.fromstring(color, np.uint8).reshape(480,640,3)

In [43]:
def testJPEG(qua):
    d=r=dd=rd=0
    for i in range(1,441):
        depth, rgb = getData(i)
        img = Image.fromarray(rgb)
        img1 = Image.fromarray(depth)
        fpath =BytesIO()
        img.save(fpath, quality = qua, format = "JPEG")
        fpath.seek(0)
        d += (len(fpath.getvalue()))
        dpath = BytesIO()
        img1.save(dpath, quality = qua, format = "JPEG")
        dpath.seek(0)
        r += (len(dpath.getvalue()))
        depth1 = np.array(Image.open(dpath))
        rgb1 = np.array(Image.open(fpath))
        rd += np.mean(rgb - rgb1)
        dd += np.mean(depth - depth1)
    d /= 440.0
    r /= 440.0
    rd /= 440.0
    dd /= 440.0
    print("Depth:"+str(qua))
    print("Average After compressed:"+str(d))
    print("Compression rate: "+str(d/307200.0*100)+"%")
    print("Average Loss : "+str(dd))
    print("Color:"+str(qua))
    print("Average After compressed:"+str(r))
    print("Compression rate: "+str(r/307200.0*100)+"%")
    print("Average Loss : "+str(rd))

In [44]:
for i in range(95, 50, -5):
    testJPEG(i)

Depth:95
Average After compressed:129644.672727
Compression rate: 42.2020419034%
Average Loss : 20.4787287302
Color:95
Average After compressed:58715.1431818
Compression rate: 19.1130023378%
Average Loss : 105.635120413
Depth:90
Average After compressed:85169.0772727
Compression rate: 27.724309008%
Average Loss : 26.2367088216
Color:90
Average After compressed:45408.4954545
Compression rate: 14.7814112808%
Average Loss : 108.132225369
Depth:85
Average After compressed:66267.5090909
Compression rate: 21.5714547822%
Average Loss : 29.9312478989
Color:85
Average After compressed:38934.5840909
Compression rate: 12.6740182588%
Average Loss : 109.442073728
Depth:80
Average After compressed:55664.3477273
Compression rate: 18.1199048591%
Average Loss : 35.1961558727
Color:80
Average After compressed:35198.0045455
Compression rate: 11.4576837713%
Average Loss : 110.529037302
Depth:75
Average After compressed:48148.9795455
Compression rate: 15.6734959458%
Average Loss : 34.0470837846
Color:75
Av